Exercise 1
Write a Python program to classify letters from different fonts using delta rule learning. The
training set contains 21 input vectors (patterns)....In other words, we train the perceptron to classify each of these vectors as belonging, or not
belonging, to the class A (or B, C, D as stated above). In that case, the target value for each
pattern is either 1 or - 1; only the first component of the target vector shown is applicable. The
net is as shown below, and n = 63
There are three examples of A and 18 examples of not-A in the training set.
a) First, classify the input using perceptron.
b) Then, classify the input using ADALINE and compare the ability of the trained
ADALINE to classify noisy input to the results for the perceptron. Try 5, 10, 15, 20
pixels wrong and the same levels of missing data
Your output should display the results of classification in a friendly format and state the
differences between two algorithms regarding handling noise and missing data.

In [7]:
#Cody Barker 301441462
#Imports!

import math
import random
import numpy as np
from typing import List, Tuple
    
# converts a list of 0s and 2s to a list of -1s and 1s, do not use as copied for doc we were given but this is the fucntion that would convert   
def bipolar(vec) -> List[int]:
    return [p - 1 for p in vec]

# converts a list of vectors to a row of integers. Flattens matrix into vector
def flatten_grid(pattern) -> List[int]:
    flat = []
    for row in pattern:
        for pix in row:
            flat.append(pix)
    return flat

# flip bits to create noise
def add_noise(vector, num_flips) -> List[int]:
    noisy = vector.copy()
    flip_indices = random.sample(range(len(noisy)), num_flips)
    for i in flip_indices:
        noisy[i] *= -1
    return noisy

# adds missing value to data
def add_missing_data(vector, num_missing) -> List[int]:
    missing = vector.copy()
    missing_indices = random.sample(range(len(missing)), num_missing)
    for i in missing_indices:
        missing[i] = 0 
    return missing
    
# make patterns for letters A, B, C, D, E, K, J, Copied from doc we were given
def make_letter_patterns():
    
    A1 = [
        [-1, -1,  1,  1, -1, -1, -1],
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1,  1,  1,  1,  1,  1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [ 1,  1,  1, -1,  1,  1,  1],
    ]
    
    A2 = [
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [-1,  1,  1,  1,  1,  1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
    ]
    A3 = [
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1, -1,  1, -1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1, -1,  1, -1,  1, -1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [-1,  1,  1,  1,  1,  1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1,  1, -1, -1, -1,  1,  1],
    ]
    
    B1 = [
        [ 1,  1,  1,  1,  1,  1, -1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1,  1,  1,  1,  1, -1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],  
        [ 1,  1,  1,  1,  1,  1, -1],  
    ]
    
    B2 = [
        [ 1,  1,  1,  1,  1,  1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1,  1,  1,  1,  1,  1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1,  1,  1,  1,  1,  1, -1],
    ]
    
    B3 = [
        [ 1,  1,  1,  1,  1,  1, -1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1,  1,  1,  1,  1, -1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [ 1,  1,  1,  1,  1,  1, -1],
    ]

    C1 = [
        [-1, -1,  1,  1,  1,  1,  1],
        [-1,  1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [-1,  1, -1, -1, -1, -1,  1],
        [-1, -1,  1,  1,  1,  1, -1],
    ]
    
    C2 = [
        [-1, -1,  1,  1,  1, -1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1,  1],
        [-1,  1, -1, -1, -1,  1, -1],
        [-1, -1,  1,  1,  1, -1, -1],
    ]

    C3 = [
    [-1, -1,  1,  1,  1, -1,  1],
    [-1,  1, -1, -1, -1,  1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1,  1,  1],
    [-1, -1,  1,  1,  1, -1, -1],
]
    
    D1 = [
    [ 1,  1,  1,  1,  1, -1, -1],
    [ 1, -1, -1, -1, -1,  1, -1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1,  1, -1],
    [ 1,  1,  1,  1,  1, -1, -1],
]

    D2 = [
    [ 1,  1,  1,  1,  1, -1, -1],
    [ 1, -1, -1, -1, -1,  1, -1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1, -1,  1],
    [ 1, -1, -1, -1, -1,  1, -1],
    [ 1,  1,  1,  1,  1, -1, -1],
]

    D3 = [
    [ 1,  1,  1,  1,  1, -1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1,  1, -1],
    [ 1,  1,  1,  1,  1, -1, -1],
]

    E1 = [
    [ 1,  1,  1,  1,  1,  1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1,  1,  1,  1,  1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1,  1,  1,  1,  1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [-1,  1, -1, -1, -1, -1,  1],
    [ 1,  1,  1,  1,  1,  1,  1],
]

    E2 = [
    [ 1,  1,  1,  1,  1,  1,  1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1,  1,  1,  1,  1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1, -1, -1, -1, -1, -1, -1],
    [ 1,  1,  1,  1,  1,  1,  1],
]
    
    E3 = [
    [ 1,  1,  1,  1,  1,  1,  1],
    [-1,  1, -1, -1, -1, -1, -1],
    [-1,  1, -1, -1, -1, -1, -1],
    [-1,  1,  1,  1,  1,  1, -1],
    [-1,  1, -1, -1, -1, -1, -1],
    [-1,  1,  1,  1,  1,  1, -1],
    [-1,  1, -1, -1, -1, -1, -1],
    [-1,  1, -1, -1, -1, -1, -1],
    [ 1,  1,  1,  1,  1,  1,  1],
]
    
    K1 = [
    [ 1,  1,  1, -1, -1,  1,  1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1,  1, -1, -1, -1, -1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1, -1, -1,  1, -1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [ 1,  1, -1, -1, -1, -1,  1],
]

    K2 = [
    [ 1, -1, -1, -1, -1,  1, -1],
    [ 1, -1, -1,  1, -1, -1, -1],
    [ 1, -1,  1, -1,  1, -1, -1],
    [ 1,  1, -1, -1,  1, -1, -1],
    [ 1,  1, -1,  1, -1, -1, -1],
    [ 1, -1,  1, -1,  1, -1, -1],
    [ 1, -1, -1,  1, -1,  1, -1],
    [ 1, -1, -1, -1,  1, -1, -1],
    [ 1, -1, -1, -1, -1,  1, -1],
]

    K3 = [
    [ 1,  1,  1, -1, -1,  1,  1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1,  1, -1, -1, -1, -1],
    [-1,  1, -1,  1, -1, -1, -1],
    [-1,  1, -1, -1,  1, -1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [ 1,  1, -1, -1, -1, -1,  1],
]

    J1 = [
    [ 1,  1,  1,  1,  1,  1,  1],
    [-1, -1, -1, -1,  1, -1, -1],
    [-1, -1, -1, -1,  1, -1, -1],
    [-1, -1, -1, -1,  1, -1, -1],
    [-1, -1, -1, -1,  1, -1, -1],
    [ 1, -1, -1, -1,  1, -1, -1],
    [ 1, -1, -1, -1,  1, -1, -1],
    [ 1, -1, -1, -1,  1, -1, -1],
    [-1,  1,  1,  1, -1, -1, -1],
]
    
    J2 = [
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [-1, -1,  1,  1,  1, -1, -1],
]
    
    J3 = [
    [-1, -1, -1, -1, -1,  1,  1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1, -1, -1, -1, -1,  1, -1],
    [-1,  1, -1, -1, -1,  1, -1],
    [-1, -1,  1,  1,  1, -1, -1],
]

    return {
    'A': [A1, A2, A3],
    'B': [B1, B2, B3],
    'C': [C1, C2, C3],
    'D': [D1, D2, D3],
    'E': [E1, E2, E3],
    'K': [K1, K2, K3],
    'J': [J1, J2, J3],
}

class Perceptron:
    #Perceptron class with methods for training and prediction
    def __init__(self, input_size: int, learning_rate: float = 0.01, epochs: int = 20):
        # initialize weights and bias
        self.weights = [random.uniform(-0.5, 0.5) for _ in range(input_size)]
        self.bias = random.uniform(-0.5, 0.5)
        self.learning_rate = learning_rate
        self.epochs = epochs
        
    def activation(self, x: float) -> int:
        return 1 if x >= 0 else -1
    
    def predict(self, x: List[int]) -> int:
        total = sum(w * xi for w, xi in zip(self.weights, x)) + self.bias
        return self.activation(total)
    
    def train(self, X: List[List[int]], y: List[int]) -> None:
        for epoch in range(self.epochs):
            for xi, target in zip(X, y):
                prediction = self.predict(xi)
                error = target - prediction
                self.weights = [w + self.learning_rate * error * xi for w, xi in zip(self.weights, xi)]
                self.bias += self.learning_rate * error
  
class Adaline:
    def __init__(self, input_size: int, learning_rate: float = 0.01, epochs: int = 20):
        self.weights = np.random.uniform(-0.5, 0.5, input_size)
        self.bias = random.uniform(-0.5, 0.5)
        self.learning_rate = learning_rate
        self.epochs = epochs
        
    def activation(self, x: float) -> int:
        return np.where(x >= 0, 1, -1)
    
    def train(self, X: List[List[int]], y: List[int]) -> None:
        for epoch in range(self.epochs):
            for xi, target in zip(X, y):
                input = np.dot(xi, self.weights) + self.bias
                error = target - input
                # weight update
                self.weights += self.learning_rate * error * np.array(xi) 
                self.bias += self.learning_rate * error
                
    def predict(self, x: List[int]) -> int:
        total = np.dot(x, self.weights) + self.bias
        return self.activation(total)
    
def part_a(patterns, train_ratio = 0.7):
    X, y, labels = [], [], []

    # flatten all patterns and assign labels
    for letter, variations in patterns.items():
        for pattern in variations:
            X.append(flatten_grid(pattern))
            y.append(1 if letter == 'A' else -1)
            labels.append(letter)

    X, y, labels = np.array(X), np.array(y), np.array(labels)

    # ensure at least 2 A's in training
    indices = list(range(len(X)))
    A_indices = [i for i, lbl in enumerate(labels) if lbl == 'A']
    notA_indices = [i for i, lbl in enumerate(labels) if lbl != 'A']

    # pick at least 2 A's for training
    chosen_A_train = random.sample(A_indices, 2)
    remaining_indices = [i for i in indices if i not in chosen_A_train]

    # shuffle the remaining indices
    random.shuffle(remaining_indices)

    # calculate how many total go into training
    split_idx = int(len(X) * train_ratio)

    # fill training set with chosen A's + rest until split
    final_train_idx = chosen_A_train + remaining_indices[:split_idx - len(chosen_A_train)]
    final_test_idx = remaining_indices[split_idx - len(chosen_A_train):]

    # build train/test sets
    train_X, test_X = X[final_train_idx], X[final_test_idx]
    train_y, test_y = y[final_train_idx], y[final_test_idx]
    train_labels, test_labels = labels[final_train_idx], labels[final_test_idx]

    # train Perceptron
    perceptron = Perceptron(input_size=train_X.shape[1], learning_rate=0.5, epochs=20)
    perceptron.train(train_X, train_y)

    # train Adaline
    adaline = Adaline(input_size=train_X.shape[1], learning_rate=0.01, epochs=150)
    adaline.train(train_X, train_y)

    # test both models
    print("Classification Results")
    correct_p, correct_a = 0, 0
    for x, true_label, letter in zip(test_X, test_y, test_labels):
        p_pred = perceptron.predict(x)
        a_pred = adaline.predict(x)
        print(f"Letter {letter}: Perceptron → {'A' if p_pred == 1 else 'Not A'}, "
              f"ADALINE → {'A' if a_pred == 1 else 'Not A'}, "
              f"True → {'A' if true_label == 1 else 'Not A'}")

        if p_pred == true_label:
            correct_p += 1
        if a_pred == true_label:
            correct_a += 1

    # print accuracy
    print("\nSummary:")
    print(f"Perceptron Accuracy: {correct_p / len(test_y) * 100:.2f}%")
    print(f"ADALINE Accuracy:    {correct_a / len(test_y) * 100:.2f}%")
    
    
def part_b(patterns):
    X=[]
    y=[]
    
    for letter, variation in patterns.items():
        for pattern in variation:
            X.append(flatten_grid(pattern))
            
            if letter == 'A':
                y.append(1)
            else:
                y.append(-1)
                
    X, y = np.array(X), np.array(y)     
    # perceptron train and test
    perceptron = Perceptron(input_size=X.shape[1], learning_rate=0.5, epochs=20)
    perceptron.train(X, y)
    
    # TRain Adaline
    adaline = Adaline(input_size=X.shape[1], learning_rate=0.01, epochs=150)
    adaline.train(X, y)
    
    clean_A = flatten_grid(patterns['A'][0])
    
    for noise_level in [5, 10, 15, 20]:
        perceptron_noise_results = []
        adaline_noise_results = []
        perceptron_missing_results = []
        adaline_missing_results = []
        trials = 10000
        
        for _ in range(trials):
            noisy = add_noise(clean_A, noise_level)
            missing = add_missing_data(clean_A, noise_level)
            perceptron_noise_results.append(perceptron.predict(noisy))
            adaline_noise_results.append(adaline.predict(noisy))
            perceptron_missing_results.append(perceptron.predict(missing))
            adaline_missing_results.append(adaline.predict(missing)) 
        
        print(f"\nNoise level {noise_level} pixels (over {trials} trials):")
        print(f"  Perceptron → 'A:'{perceptron_noise_results.count(1)}, 'Not A:'{perceptron_noise_results.count(-1)}")
        print(f"  ADALINE    → 'A:'{adaline_noise_results.count(1)}, 'Not A:'{adaline_noise_results.count(-1)}")
        print(f"Missing {noise_level} pixels (over {trials} trials):")
        print(f"  Perceptron → 'A:'{perceptron_missing_results.count(1)}, 'Not A:'{perceptron_missing_results.count(-1)}")
        print(f"  ADALINE    → 'A:'{adaline_missing_results.count(1)}, 'Not A:'{adaline_missing_results.count(-1)}")
            
if __name__ == '__main__':
    patterns = make_letter_patterns()
    part_a(patterns)
    part_b(patterns)

Classification Results
Letter K: Perceptron → Not A, ADALINE → Not A, True → Not A
Letter B: Perceptron → Not A, ADALINE → Not A, True → Not A
Letter D: Perceptron → Not A, ADALINE → Not A, True → Not A
Letter E: Perceptron → Not A, ADALINE → Not A, True → Not A
Letter B: Perceptron → Not A, ADALINE → Not A, True → Not A
Letter C: Perceptron → Not A, ADALINE → A, True → Not A
Letter D: Perceptron → Not A, ADALINE → Not A, True → Not A

Summary:
Perceptron Accuracy: 100.00%
ADALINE Accuracy:    85.71%

Noise level 5 pixels (over 10000 trials):
  Perceptron → 'A:'9992, 'Not A:'8
  ADALINE    → 'A:'8082, 'Not A:'1918
Missing 5 pixels (over 10000 trials):
  Perceptron → 'A:'10000, 'Not A:'0
  ADALINE    → 'A:'9648, 'Not A:'352

Noise level 10 pixels (over 10000 trials):
  Perceptron → 'A:'9658, 'Not A:'342
  ADALINE    → 'A:'7062, 'Not A:'2938
Missing 10 pixels (over 10000 trials):
  Perceptron → 'A:'10000, 'Not A:'0
  ADALINE    → 'A:'8949, 'Not A:'1051

Noise level 15 pixels (over 10000 

We can see from the results and run again multiple times to confirm. Perceptron uses a hard threshold acitvation, as long as ther weighted sum crossse the threshold the exact magnitude does not matter. Therefore it can handle losing a few pixels, and can tolerate a bit of noise, but fails when the noise is strong enough to flip it. Adaline uses delta rule and considers the magnitude of the acitvation before applying the step function. This allows for more precise weights during training, but even small noise or missing pixels shift the weighted sum, as the model is mor e senstive. I a test I did, for 5 pixels missing or noise perceptron as still able to identify most correct values, however as more and more nosie or missin pixels were added the percentage of correct values was lowered. For Adaline even for low amounts of noise or missing values it did not have as good of a correct percentage as Perceptron. But Adaline handled noise and missing values similar with similar correct percentages, while perceptron handled noise better tha it is able to handle missing vlaues. Perceptron is better at handling disortions, while adaline is more sensitive, however it provides a more prinicpled learnign rule that works better when the data is consistent.